# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [22]:
import numpy as np
%load_ext autoreload
%autoreload 2
from ExchangeEconomy import ExchangeEconomyClass
model = ExchangeEconomyClass()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Question 1

**Explain how you solve the model**

# Question 2

Explain your code and procedure

In [28]:
#First we define price for good 1
p1 = [(0.5 + 2*i/75) for i in range(76)]

#Here we make calculations for errors in market for good 1 and 2
errors = [model.check_market_clearing(x) for x in p1]
eps1 = [x[0] for x in errors]
eps2 = [x[1] for x in errors]

#We import panda to create DataFramce
import pandas as pd

#Here we create DataFrame 
df = pd.DataFrame(errors, columns=['eps1', 'eps2'])
print(df)

        eps1      eps2
0  -0.133333 -0.433333
1  -0.156962 -0.417333
2  -0.178313 -0.401333
3  -0.197701 -0.385333
4  -0.215385 -0.369333
..       ...       ...
71 -0.502507  0.702667
72 -0.503581  0.718667
73 -0.504632  0.734667
74 -0.505660  0.750667
75 -0.506667  0.766667

[76 rows x 2 columns]


# Question 3

Explain your code and procedure

In [31]:
#In question 3 we find the market clearing price 1 (when price 2=1).
#we can see in question 2 that the price is somewhere between 0,5 and 2,5
#First we define price for good 1
p1 = [(0.5 + 2*i/1000) for i in range(1001)]

# Iterate over the eps1 values to find when error = 0
for i, error in enumerate(eps1):
    if abs(error) < 0.2:  # Adjust the tolerance as needed
        p1_with_error_zero = p1[i]
        print("p1 value that makes error = 0:", p1_with_error_zero)
        break
else:
    print("No p1 value found that makes error = 0")

p1 value that makes error = 0: 0.5


ADD CONCISE CONLUSION.